In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from strategy_v2.Strategy import *
from strategy_v2.TradingSubSystem import *
from strategy_v2.Portfolio import *
from strategy_v2.TransactionCost import *
from strategy_v2.Executor import *
from utils.data_helper import *
from utils.data import *
from utils.performance import *
import cvxpy as cp
from plotly.subplots import make_subplots
from croniter import croniter

# Portfolio General Settings
We assume the code is run at T before market open, so it all data on and before T-1 close. Therefore, the end_date should be today - BDay(1)

In [2]:
end_date = get_today()
start_date = pd.to_datetime(datetime(2022, 1, 3))
vol_target = 0.5
max_leverage = 1

# Linear Optimization (Risk-Adjusted Return)

- How to determine the capital?

Difficulty here is portfolio might use different leverage per rebalance, looking at the asset MV might not give you the total portfolio capital (e.g. 70% leverage, your stocks only worth 70% of your capital now)

we can use the market value on Futu divides by the leverage used for last periods. This will give you the total captial for the portfolio

Then we use this capital for next rebalance

We shuold strategically fix this issue by reading capital from Futu or keep track the capital ourselves

- 2024-03-23 - Leverage Ratio = 102.4376
- 2024-04-20 - Leverage Ratio = 102.6119

In [5]:
176443.23/102.4376*100

172244.5957343788

In [6]:
portfolio = PortfolioLinearOpt(
    capital=172244.5957343788,  
    name='ETF Buy and Hold Portfolio',
    lookback_period=60,
    opt_freq=1,
    rebalance_iter=RebalancerIter('0 0 * * Fri', 2),    
    tc_model=TransactionCostFutu(),    
    systems=[
        TradingSubSystemSingle(vol_target=vol_target, instruments=['META'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['TSLA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['NVDA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['AAPL'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['EWY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['DXJ'],   strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),        
        #TradingSubSystemSingle(vol_target=vol_target, instruments=['VOE'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        #TradingSubSystemSingle(vol_target=vol_target, instruments=['VBR'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['BRK-B'], strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['SPY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
        TradingSubSystemSingle(vol_target=vol_target, instruments=['QQQ'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),        
])

portfolio.set_start_date(start_date)
portfolio.set_end_date(end_date)
portfolio.backtest_subsystems()
portfolio.optimize(gamma=10, hhi=0.2)
portfolio.backtest()
portfolio.rebalance()
portfolio.performance(show_all_rets=True)

2024-04-20 01:20:59,761 - TradingSubSystemSingle [META] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-04-19......
2024-04-20 01:20:59,767 - TradingSubSystemSingle [META] - INFO - Volatility Target = 50.0% | Price Volatility = 26.3% | Last Scale Factor = 2.14
2024-04-20 01:20:59,768 - TradingSubSystemSingle [TSLA] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-04-19......
2024-04-20 01:20:59,773 - TradingSubSystemSingle [TSLA] - INFO - Volatility Target = 50.0% | Price Volatility = 44.0% | Last Scale Factor = 1.13
2024-04-20 01:20:59,777 - TradingSubSystemSingle [NVDA] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-04-19......
2024-04-20 01:20:59,782 - TradingSubSystemSingle [NVDA] - INFO - Volatility Target = 50.0% | Price Volatility = 38.4% | Last Scale Factor = 1.38
2024-04-20 01:20:59,786 - TradingSubSystemSingle [AAPL] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-04-19

2024-04-20 01:20:59,813 - TradingSubSystemSingle [DXJ] - INFO - Volatility Target = 50.0% | Price Volatility = 13.1% | Last Scale Factor = 3.65
2024-04-20 01:20:59,817 - TradingSubSystemSingle [BRK-B] - INFO - Generating position for strategy BAH2 between 2021-10-04 and 2024-04-19......
2024-04-20 01:20:59,824 - TradingSubSystemSingle [BRK-B] - INFO - Volatility Target = 50.0% | Price Volatility = 12.1% | Last Scale Factor = 4.35
2024-04-20 01:20:59,828 - TradingSubSystemSingle [SPY] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-04-19......
2024-04-20 01:20:59,833 - TradingSubSystemSingle [SPY] - INFO - Volatility Target = 50.0% | Price Volatility = 10.5% | Last Scale Factor = 4.67
2024-04-20 01:20:59,837 - TradingSubSystemSingle [QQQ] - INFO - Generating position for strategy BAH1 between 2021-10-04 and 2024-04-19......
2024-04-20 01:20:59,842 - TradingSubSystemSingle [QQQ] - INFO - Volatility Target = 50.0% | Price Volatility = 14.2% | Last Scale Factor =

,Rebalanced Portfolio,Optimized Portfolio,^SPX,Single - META (BAH1),Single - TSLA (BAH1),Single - NVDA (BAH1),Single - AAPL (BAH1),Single - EWY (BAH1),Single - DXJ (BAH2),Single - BRK-B (BAH2),Single - SPY (BAH1),Single - QQQ (BAH1)
Measure,,,,,,,,,,,,
Cumulative Return,1.593584,1.587415,1.040110,1.528210,0.452989,2.811434,0.941570,0.822680,1.809760,1.350117,1.084116,1.066346
Annualized Return,0.227215,0.225320,0.034465,0.309860,-0.223329,0.567991,0.013346,-0.054884,0.272615,0.147201,0.052628,0.059324
Annualized Volatility,0.218164,0.217182,0.185886,0.493956,0.493077,0.487527,0.281793,0.246511,0.163808,0.179376,0.186223,0.249997
Annualized Sharpe Ratio,0.879697,0.874949,-0.004474,0.555845,-0.524514,1.092647,-0.077897,-0.365828,1.448755,0.623851,0.093064,0.096111
Maximum Drawdown,-0.263013,-0.271779,-0.254251,-0.682479,-0.635372,-0.531787,-0.309294,-0.406293,-0.117846,-0.265790,-0.244964,-0.348280


# Equal Weighted Portfolios

In [7]:
# portfolio = PortfolioStandard(    
#     capital=200000,
#     name='ETF Buy and Hold Portfolio',    
#     rebalance_iter=RebalancerIter('0 0 * * Fri', 2),
#     tc_model=TransactionCostFutu(),    
#     systems=[
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['META'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['TSLA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['NVDA'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         # TradingSubSystemSingle(vol_target=vol_target, instruments=['AAPL'],  strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['EWY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['DXJ'],   strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),        
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['VOE'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['VBR'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['BRK-B'], strategy=[BuyAndHoldStrategy(confidence=2)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['SPY'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),
#         TradingSubSystemSingle(vol_target=vol_target, instruments=['QQQ'],   strategy=[BuyAndHoldStrategy(confidence=1)], max_leverage=max_leverage),       
# ])

# portfolio.set_start_date(start_date)
# portfolio.set_end_date(end_date)
# portfolio.backtest_subsystems()
# portfolio.optimize()
# portfolio.backtest()
# portfolio.rebalance()
# portfolio.performance()

# Execute the Portfolio

In [10]:
executor = ExecutorFutu(is_test=False)
executor.set_portfolio(portfolio)
orders = executor.execute(px_interval='5m')
orders

2024-04-20 01:22:35,180 - ExecutorFutu - INFO - market: US
2024-04-20 01:22:35,185 - ExecutorFutu - INFO - Cancel all orders first before executing.....


2024-04-20 01:22:35,188 | 44206 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=13, host=127.0.0.1, port=11111, user_id=18214795
2024-04-20 01:22:35,310 | 44206 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=13


2024-04-20 01:22:38,310 - Linear Optimized Portfolio (ETF Buy and Hold Portfolio) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $172,245
2024-04-20 01:22:38,311 - Linear Optimized Portfolio (ETF Buy and Hold Portfolio) - INFO - Generate trade position based on target capital of $172,245
2024-04-20 01:22:38,317 - ExecutorFutu - INFO - Execute Linear Optimized Portfolio (ETF Buy and Hold Portfolio) position based on 2024-04-19


2024-04-20 01:22:38,320 | 44206 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=14, host=127.0.0.1, port=11111, user_id=18214795


2024-04-20 01:22:38,332 - Futu - INFO - 8 Positions: US.SPY, US.QQQ, US.NVDA, US.NFLX, US.META, US.EWY, US.DXJ, US.BRK.B


...

2024-04-20 01:22:55,076 - Futu - INFO - Placed Order: {'code': 'US.SPY', 'price': 497.72, 'qty': 1.0, 'trd_side': 'SELL', 'order_type': 'NORMAL', 'market': 'US', 'trd_env': 'REAL'}


2024-04-20 01:22:55,078 | 44206 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=20
2024-04-20 01:22:58,088 | 44206 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=21, host=127.0.0.1, port=11111, user_id=18214795


2024-04-20 01:22:58,407 - Futu - INFO - Placed Order: {'code': 'US.QQQ', 'price': 417.82, 'qty': 15.0, 'trd_side': 'SELL', 'order_type': 'NORMAL', 'market': 'US', 'trd_env': 'REAL'}


2024-04-20 01:22:58,408 | 44206 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=21


,code,stock_name,trd_side,order_type,order_status,order_id,qty,price,create_time,updated_time,...,remark,time_in_force,fill_outside_rth,aux_price,trail_type,trail_value,trail_spread,currency,portfolio,date
0,US.META,Meta Platforms,SELL,NORMAL,SUBMITTING,8991437020360605778,1.0,484.80,2024-04-19 13:22:38,2024-04-19 13:22:38,...,Linear Optimized Portfolio (ETF Buy and Hold P...,DAY,False,N/A,N/A,N/A,N/A,USD,Linear Optimized Portfolio (ETF Buy and Hold P...,2024-04-19
0,US.NVDA,英伟达,SELL,NORMAL,SUBMITTING,136886683035226196,21.0,811.45,2024-04-19 13:22:42,2024-04-19 13:22:42,...,Linear Optimized Portfolio (ETF Buy and Hold P...,DAY,False,N/A,N/A,N/A,N/A,USD,Linear Optimized Portfolio (ETF Buy and Hold P...,2024-04-19
0,US.EWY,韩国ETF-iShares MSCI,BUY,NORMAL,SUBMITTING,5674618580285634449,115.0,61.65,2024-04-19 13:22:45,2024-04-19 13:22:45,...,Linear Optimized Portfolio (ETF Buy and Hold P...,DAY,False,N/A,N/A,N/A,N/A,USD,Linear Optimized Portfolio (ETF Buy and Hold P...,2024-04-19
0,US.DXJ,日本对冲股票ETF-WisdomTree,BUY,NORMAL,SUBMITTING,607322255621658487,52.0,105.48,2024-04-19 13:22:48,2024-04-19 13:22:48,...,Linear Optimized Portfolio (ETF Buy and Hold P...,DAY,False,N/A,N/A,N/A,N/A,USD,Linear Optimized Portfolio (ETF Buy and Hold P...,2024-04-19
0,US.BRK.B,伯克希尔-B,BUY,NORMAL,SUBMITTING,2451165752272682741,19.0,403.58,2024-04-19 13:22:51,2024-04-19 13:22:51,...,Linear Optimized Portfolio (ETF Buy and Hold P...,DAY,False,N/A,N/A,N/A,N/A,USD,Linear Optimized Portfolio (ETF Buy and Hold P...,2024-04-19
0,US.SPY,SPDR 标普500指数ETF,SELL,NORMAL,SUBMITTING,8269427348457861859,1.0,497.72,2024-04-19 13:22:55,2024-04-19 13:22:55,...,Linear Optimized Portfolio (ETF Buy and Hold P...,DAY,False,N/A,N/A,N/A,N/A,USD,Linear Optimized Portfolio (ETF Buy and Hold P...,2024-04-19
0,US.QQQ,纳指100ETF-Invesco QQQ Trust,SELL,NORMAL,SUBMITTING,8235876018767211736,15.0,417.82,2024-04-19 13:22:58,2024-04-19 13:22:58,...,Linear Optimized Portfolio (ETF Buy and Hold P...,DAY,False,N/A,N/A,N/A,N/A,USD,Linear Optimized Portfolio (ETF Buy and Hold P...,2024-04-19
